In [6]:
from __future__ import print_function

import numpy as np
import pandas as pd
np.random.seed(1)

import sys
import sklearn
import sklearn.ensemble
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

import shap
from int_met import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
dataset_folder = 'Datasets/drugscom/'
train_raw = pd.read_csv(str(dataset_folder+'drugsComTrain_raw.csv'))
test_raw = pd.read_csv(str(dataset_folder+'drugsComTest_raw.csv'))

In [8]:
train = train_raw[['review','rating']]
test = train_raw[['review','rating']]
df_all = pd.concat([train,test]).reset_index()
del df_all['index']
del train
del test

In [9]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

In [10]:
stops = set(stopwords.words('english'))
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    if i in stops:
        stops.remove(i)
    
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    return( ' '.join(stemming_words))

In [11]:
%time df_all['review_clean'] = df_all['review'].apply(review_to_words)

Wall time: 5min 3s


In [12]:
df_all['sentiment'] = df_all["rating"].apply(lambda x: 1 if x > 5 else 0)
df_all.drop_duplicate(inplace = True)

In [13]:
df_train, df_test = train_test_split(df_all, test_size=0.33, random_state=42)
del df_all

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2,
                             max_features = 5000
                            )

In [15]:
%time X_train = vectorizer.fit_transform(df_train['review_clean'])
%time X_test = vectorizer.transform(df_test['review_clean'])

Wall time: 12.9 s
Wall time: 4.69 s


In [16]:
y_train = df_train['sentiment']
y_test = df_test['sentiment']
del df_train
del df_test

In [17]:
import pickle
pickling_on = open("textrf.pickle","rb")
c = pickle.load(pickling_on)
pickling_on.close()

In [18]:
#c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=1)
#c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 0.9998704525370483
Test 0.968156156945997


In [ ]:
a = vectorizer.transform(['abl']).toarray()

In [ ]:
a[0,:8]

In [ ]:
vectorizer.vocabulary_ 

In [19]:
X_trainn = pd.DataFrame(data = X_train.toarray())
X_testt = pd.DataFrame(data = X_test.toarray())
#del X_train
#del X_test

In [20]:
med = X_trainn.median()

In [22]:
import pickle
ff = open("text_median.pickle","wb")
pickle.dump(med, ff)
#med = pickle.load(ff)
ff.close()

In [23]:
explainer = shap.KernelExplainer(predict_fn, med)

In [30]:
samples = 400

In [46]:
exp = explainer.shap_values(X_testt[0:samples])
exp2 = explainer.shap_values(X_testt[0:samples])

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [05:34<00:00,  1.62it/s]


In [56]:
i = calc_identity(exp, exp2)
print(i)

(88.0, 48, 400)


In [49]:
X_testt.head()

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
sb = calc_stability(exp, y_test.iloc[0:samples])
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(60, 400)


In [54]:
s = calc_separability(exp2)
print(s)

(1190, 400, 160000, 0.74375)
